[toc]

# Tensorflow2 利用 lstm 实现股票预测

## 导入数据

In [1]:
import tensorflow as tf
import numpy as np
import tushare as ts
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

seq_length = 20
input_dim = 5
output_dim = 1

stock_data = ts.get_k_data('600000', start='2016-01-01', end='2018-11-20')
features = ['open', 'high', 'low', 'volume', 'close']
target = ['close']
data = stock_data[features].values # shape = (685, 5)
target = stock_data[target].values # shape = (685, 1)

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


## 划分验证集

这里不用 sklearn.model_selection.train_test_split，因为这会导致数据穿越

In [2]:
val_rate = 0.1
sample_size = data.shape[0]
train_size = int(sample_size * (1 - val_rate))
x_train, x_val, y_train, y_val = data[:train_size], data[train_size:], target[:train_size], target[:train_size]

print(f"train size: {train_size}, val sizse: {sample_size - train_size}")

train size: 616, val sizse: 69


## 对数据进行标准化

In [3]:
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

x_train_scaled = x_scaler.fit_transform(x_train)
y_train_scaled = y_scaler.fit_transform(y_train)
x_val_scaled = x_scaler.transform(x_val)
y_val_scaled = y_scaler.transform(y_val)

## 取得序列数据

In [4]:
def get_sequence_data(x, y):
    n = x.shape[0]
    sample_size = n - (seq_length + 1)
    x_sequence_sample = np.array([x[i:i+seq_length] for i in range(sample_size)])
    y_sequence_sample = np.array([y[i+seq_length] for i in range(sample_size)])
    return x_sequence_sample, y_sequence_sample

In [5]:
x_train_sequence, y_train_sequence = get_sequence_data(x_train_scaled, y_train_scaled)
x_val_sequence, y_val_sequence = get_sequence_data(x_val_scaled, y_val_scaled)

print(f"x_train_sequence shape: {x_train_sequence.shape}, y_train_sequence shape: {y_train_sequence.shape}")
print(f"x_val_sequence shape: {x_val_sequence.shape}, y_val_sequence shape: {y_val_sequence.shape}")

x_train_sequence shape: (595, 20, 5), y_train_sequence shape: (595, 1)
x_val_sequence shape: (48, 20, 5), y_val_sequence shape: (48, 1)


## 搭建网络

In [7]:
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model

# 构建神经网络层 1层LSTM层+3层Dense层
lstm_input = Input(shape=(seq_length, input_dim), name='lstm_input') # input层不包括batch的shape
lstm_output = LSTM(128, activation='tanh', dropout=0.5)(lstm_input)  # LSTM网络
Dense_output_1 = Dense(64, activation='relu')(lstm_output)  
Dense_output_2 = Dense(16, activation='relu')(Dense_output_1)  
predictions = Dense(output_dim, activation='tanh')(Dense_output_2) 
model = Model(inputs=lstm_input, outputs=predictions)
def customLoss(y, yhat):
    return tf.reduce_mean((tf.math.abs(y - yhat)/(y+1e-7)))

earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=customLoss)
model.fit(x_train_sequence, y_train_sequence, validation_data=(x_val_sequence, y_val_sequence), batch_size=32, epochs=1000, callbacks=[earlyStopping])

Train on 595 samples, validate on 48 samples
Epoch 1/1000
595/595 [==============================] - 2s 3ms/sample - loss: 3656.6171 - val_loss: 0.6933
Epoch 2/1000
595/595 [==============================] - 0s 464us/sample - loss: 547.4767 - val_loss: 0.7901
Epoch 3/1000
595/595 [==============================] - 0s 355us/sample - loss: 172.6736 - val_loss: 0.9487
Epoch 4/1000
595/595 [==============================] - 0s 354us/sample - loss: 5.0123 - val_loss: 1.0047
Epoch 5/1000
595/595 [==============================] - 0s 392us/sample - loss: 42.2900 - val_loss: 0.7668
Epoch 6/1000
595/595 [==============================] - 0s 350us/sample - loss: 1852.0115 - val_loss: 0.9010
Epoch 7/1000
595/595 [==============================] - 0s 356us/sample - loss: 68.3455 - val_loss: 0.8221
Epoch 8/1000
595/595 [==============================] - 0s 341us/sample - loss: 732.2887 - val_loss: 0.8849
Epoch 9/1000
595/595 [==============================] - 0s 357us/sample - loss: 68.2523 - val_l

595/595 [==============================] - 0s 374us/sample - loss: 48.9200 - val_loss: 0.8697
Epoch 77/1000
595/595 [==============================] - 0s 383us/sample - loss: 15.9667 - val_loss: 0.8698
Epoch 78/1000
595/595 [==============================] - 0s 388us/sample - loss: 50.5188 - val_loss: 0.8679
Epoch 79/1000
595/595 [==============================] - 0s 404us/sample - loss: 9.3886 - val_loss: 0.8619
Epoch 80/1000
595/595 [==============================] - 0s 394us/sample - loss: 50.8406 - val_loss: 0.8575
Epoch 81/1000
595/595 [==============================] - 0s 371us/sample - loss: 13.9384 - val_loss: 0.8509
Epoch 82/1000
595/595 [==============================] - 0s 366us/sample - loss: 54.8967 - val_loss: 0.8436
Epoch 83/1000
595/595 [==============================] - 0s 376us/sample - loss: 6.0691 - val_loss: 0.8372
Epoch 84/1000
595/595 [==============================] - 0s 428us/sample - loss: 55.1786 - val_loss: 0.8311
Epoch 85/1000
595/595 [=====================

595/595 [==============================] - 0s 392us/sample - loss: 25.9973 - val_loss: 0.4696
Epoch 152/1000
595/595 [==============================] - 0s 466us/sample - loss: 26.8308 - val_loss: 0.4732
Epoch 153/1000
595/595 [==============================] - 0s 393us/sample - loss: 31.9241 - val_loss: 0.4729
Epoch 154/1000
595/595 [==============================] - 0s 422us/sample - loss: 21.4038 - val_loss: 0.4753
Epoch 155/1000
595/595 [==============================] - 0s 438us/sample - loss: 27.3173 - val_loss: 0.4683
Epoch 156/1000
595/595 [==============================] - 0s 383us/sample - loss: 30.2271 - val_loss: 0.4741
Epoch 157/1000
595/595 [==============================] - 0s 393us/sample - loss: 21.7210 - val_loss: 0.4750
Epoch 158/1000
595/595 [==============================] - 0s 379us/sample - loss: 32.1272 - val_loss: 0.4721
Epoch 159/1000
595/595 [==============================] - 0s 386us/sample - loss: 3.9143 - val_loss: 0.4825
Epoch 160/1000
595/595 [===========

595/595 [==============================] - 0s 626us/sample - loss: 48.8689 - val_loss: 1.0009
Epoch 227/1000
595/595 [==============================] - 0s 653us/sample - loss: 24.6810 - val_loss: 0.9985
Epoch 228/1000
595/595 [==============================] - 0s 595us/sample - loss: 40.8342 - val_loss: 1.0001
Epoch 229/1000
595/595 [==============================] - 0s 611us/sample - loss: 24.3809 - val_loss: 0.9991
Epoch 230/1000
595/595 [==============================] - 0s 651us/sample - loss: 28.4437 - val_loss: 1.0015
Epoch 231/1000
595/595 [==============================] - 0s 660us/sample - loss: 26.3264 - val_loss: 0.9982
Epoch 232/1000
595/595 [==============================] - 0s 795us/sample - loss: 36.8479 - val_loss: 1.0007
Epoch 233/1000
595/595 [==============================] - 0s 647us/sample - loss: 21.7039 - val_loss: 0.9970
Epoch 234/1000
595/595 [==============================] - 0s 744us/sample - loss: 41.3781 - val_loss: 1.0000
Epoch 235/1000
595/595 [==========

## 查看在 training data 上的效果

In [ ]:
y_pred = y_scaler.inverse_transform(model.predict(x_train_sequence))
y_train_truth = y_scaler.inverse_transform(y_train_sequence)
plt.plot(y_pred, label="prediction")
plt.plot(y_train_truth, label="ground truth")
plt.legend()

## 查看在 validation data 上的效果

In [ ]:
y_pred = y_scaler.inverse_transform(model.predict(x_val_sequence))
y_val_truth = y_scaler.inverse_transform(y_val_sequence)
plt.plot(y_pred, label="prediction")
plt.plot(y_val_truth, label="ground truth")
plt.legend()

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tushare as ts
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

#参数设置/parameter setting
timesteps = seq_length = 20 #时间窗/window length
data_dim = 5 #输入数据维度/dimension of input data
output_dim = 1 #输出数据维度/dimension of output data

#数据准备/data preparation 
#变量选取Open,High,Low,Close,Volume，以浦发银行股票为例
stock_data = ts.get_k_data('600000',start='2016-01-01',end='2018-11-20')
xy = stock_data[['open','close','high','low','volume']]
xy = np.array(xy.values)

#切分训练集合测试集/split to train and testing
train_size = int(len(xy) * 0.9)
test_size = len(xy) - train_size
xy_train, xy_test = np.array(xy[0:train_size]),np.array(xy[train_size:len(xy)])

#对training set进行预处理
scaler = MinMaxScaler()
xy_train_new = scaler.fit_transform(xy_train)
x_new = xy_train_new[:,0:5]
y_new = xy_train_new[:,1]

x = x_new
y = y_new
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)

#处理数据shape,准备进入神经网络层
x_real = np.vstack(dataX).reshape(-1,seq_length,data_dim)
y_real= np.vstack(dataY).reshape(-1,output_dim)
print(x_real.shape)
print(y_real.shape)
dataX = x_real
dataY = y_real

trainX, trainY = dataX, dataY

#对test set进行预处理，这里用了training的scaler
xy_test_new = scaler.transform(xy_test)
x_new = xy_test_new[:,0:5]
y_new = xy_test_new[:,1]

x = x_new
y = y_new
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next price change
    print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)

#处理数据shape,准备进入神经网络层
x_real = np.vstack(dataX).reshape(-1,seq_length,data_dim)
y_real= np.vstack(dataY).reshape(-1,output_dim)
print(x_real.shape)
print(y_real.shape)
dataX = x_real
dataY = y_real

testX, testY = dataX, dataY

from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model

# 构建神经网络层 1层LSTM层+3层Dense层
lstm_input = Input(shape=(seq_length, data_dim), name='lstm_input')#shape: 形状元组（整型）不包括batch size。表示了预期的输入将是一批（seq_len,data_dim）的向量。
lstm_output = LSTM(128, activation='tanh', dropout=0.5)(lstm_input)#LSTM网络
#units: Positive integer,dimensionality of the output space.
#dropout: Float between 0 and 1. Fraction of the units to drop for the linear transformation of the inputs.
Dense_output_1 = Dense(64, activation='relu')(lstm_output)#全连接网络
Dense_output_2 = Dense(16, activation='relu')(Dense_output_1)#全连接网络
predictions = Dense(output_dim, activation='tanh')(Dense_output_2)#全连接网络

model = Model(inputs=lstm_input, outputs=predictions)
#This model will include all layers required in the computation of output given input.
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
#Configures the model for training.
#optimizer: String (name of optimizer) or optimizer instance. See optimizers.
#loss: String (name of objective function) or objective function.The loss value will be minimized by the model.
#metrics: List of metrics to be evaluated by the model during training and testing. Typically you will use  metrics=['accuracy'].
model.fit(trainX, trainY, batch_size=len(trainX), epochs=100, verbose=2)
#Trains the model for a given number of epochs (iterations on a dataset).
#verbose: Integer. 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.

# 保存模型
model.save('model.h5')


查看在 training data 上的效果

In [ ]:
trainPredict = model.predict(trainX)
trainPredict1 = trainPredict * scaler.data_range_[1] + scaler.data_min_[1]
trainY1 = trainY * scaler.data_range_[1] + scaler.data_min_[1]
plt.plot(trainY1,color='blue', label='ground truth')
plt.plot(trainPredict1,color='orange', label="prediction")
plt.legend()
plt.title("train")
plt.show()
print(max(abs(trainPredict1 - trainY1)))

## 预测

In [ ]:


# 载入模型
from tensorflow.keras.models import load_model
model = load_model('model.h5')

testPredict = model.predict(testX)
testPredict1 = testPredict * scaler.data_range_[1] + scaler.data_min_[1]
testY1 = testY * scaler.data_range_[1] + scaler.data_min_[1]
plt.plot(testY1,color='blue', label="ground truth")
plt.plot(testPredict1,color='orange', label="prediction")
plt.legend()
plt.title("val")
plt.show()
print(max(abs(testPredict1 - testY1)))